# Ensemble

## Ensemble Models
* ResNet152 CELoss Adam epoch 25
* EfficientNet b4 CELoss Adam epoch 25
* EfficientNet b2 CELoss Adam epoch 25

## Transforms
* BaseAugmentation
* CenterCropAugmentation

In [5]:
import os
import pandas as pd
from PIL import Image
import numpy as np

import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

#from MaskModules.transform import *
from importlib import import_module
from IPython.display import Audio

from sklearn.ensemble import RandomForestClassifier, VotingClassifier


In [6]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [7]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [14]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = getattr(import_module('MaskModules.transform'), "CenterCropAugmentation")() #TODO: Dataset 설정 변경 시 수정
dataset = TestDataset(image_paths, transform)

#TODO: 모델 별로 augmentation 다르게 주기

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)

model_paths = [
    '/opt/ml/spring_code/model_centercrop/EfficientNet_Model_epoch_25/best.pth',
    '/opt/ml/spring_code/model_centercrop/EfficientNet_B2_Model_epoch_25/best.pth',
    '/opt/ml/spring_code/model_centercrop/ResNet_Model_epoch_25/best.pth',
]
models = [torch.load(model_path) for model_path in model_paths]

#print(models)

device = torch.device('cuda')
#model = MyModel(num_classes=18).to(device)
#model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
preds = []
for model in models:
    curr_model = model.to(device)
    curr_model.eval()
    tmp_preds = []
    for images in loader:
        with torch.no_grad():
            images = images.to(device)
            pred = curr_model(images)
            tmp_preds.extend(pred.cpu().numpy())
    preds.append(tmp_preds)

In [33]:
tmp_preds = np.array(preds).T
print(tmp_preds.shape)

(18, 12600, 3)


In [39]:
all_predictions = tmp_preds.sum(axis=2)
submission['ans'] = all_predictions.argmax(axis=0)

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join('/opt/ml/spring_code/output', 'submission_ensemble.csv'), index=False)
Audio("/opt/ml/alarm.mp3", autoplay=True)
#print('test inference is done!')

In [47]:
weights = [8, 1, 1]
print(tmp_preds.dot(np.array(weights)).argmax(axis=0).shape)
submission['ans'] = tmp_preds.dot(np.array(weights)).argmax(axis=0)
submission.to_csv(os.path.join('/opt/ml/spring_code/output', 'submission_ensemble.csv'), index=False)

(12600,)


In [8]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
        transforms.Resize((224, 224), Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
    ])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)

model_path = '/opt/ml/spring_code/model_tmp/ResNext_Model_epoch_5/best.pth'

model = torch.load(model_path)

device = torch.device('cuda')
#model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions


# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join('/opt/ml/spring_code/output', 'submission_{}.csv'.format(model_path.split("/")[-2])), index=False)
Audio("/opt/ml/alarm.mp3", autoplay=True)
#print('test inference is done!')
